In [1]:
from src.Products import Products, prepare_prod_batch
from src.Reviews import Reviews
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import torch.nn as nn

Sample Batch:

In [2]:
domain = 'AMAZON_FASHION'
p = Products(domain)
r = Reviews(domain)
r.train_test_split(0.2)
batch_size = 32

In [3]:
#rerun this cell to resample
train_idx_batch = r.get_batch_bikey(batch_size, from_train=True)
train_prod_idx_batch = [p[1] for p in train_idx_batch]
train_prod_batch = [p.get_product(idx) for idx in train_prod_idx_batch]

Import ProductTower:

In [4]:
from src.product_model import ProductTower

Example of pretrained embedding layer: (Compatible with models provided by HuggingFace)

In [5]:
embed_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
embed_model = DistilBertModel.from_pretrained('distilbert-base-cased')
embed_dim = 768
seq_len = 100 #this is model independent, up to user how to truncate/pad sequences

In [6]:
def embed_model_wrapped(text):
    return embed_model(text)[0]

In [7]:
text, bop = prepare_prod_batch(train_prod_batch, embed_tokenizer)

RNN and FM params:

In [8]:
rnn_hidden_dim = 200
rnn_num_layers = 1

In [9]:
fm_n = 2000 #nfeatures
fm_embed_dim = 16

Initialize product tower and forward pass:

In [10]:
producttower = ProductTower(embed_model_wrapped, embed_dim, rnn_hidden_dim, rnn_num_layers, rnn_type='GRU', fm_type='fm', fm_field_dims=[2]*fm_n, fm_embed_dim=fm_embed_dim)

In [11]:
rnn_out, fm_out = producttower(text, bop)

In [12]:
rnn_out.shape

torch.Size([32, 100, 200])

In [13]:
fm_out.shape

torch.Size([32])